In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-tp-19011802/sample_submission.csv
/kaggle/input/2022-ai-tp-19011802/y_train.csv
/kaggle/input/2022-ai-tp-19011802/x_train.csv
/kaggle/input/2022-ai-tp-19011802/test.csv


# 디펜스 코드와의 차이점
* 데이터 전처리 시 사용 함수를 MinMaxScaler에서 StandardScaler로 바꾸어 사용함
* layer의 개수를 7개로 증가시켜줌
* Dropout 함수를 사용함

# 1) 랜덤시드 고정

In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
import torch.nn as nn
import torch.optim as optim

# 2) Train data, Test data 파일 불러오기

In [4]:
x_train=pd.read_csv('/kaggle/input/2022-ai-tp-19011802/x_train.csv')
train_y=pd.read_csv('/kaggle/input/2022-ai-tp-19011802/y_train.csv')
x_test=pd.read_csv('/kaggle/input/2022-ai-tp-19011802/test.csv')
submit=pd.read_csv('/kaggle/input/2022-ai-tp-19011802/sample_submission.csv')

In [5]:
x_train

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,white,6.8,0.43,0.26,5.2,0.043,40.0,176.0,0.99116,3.17,0.41,12.3
1,white,6.4,0.27,0.45,8.3,0.050,52.0,196.0,0.99550,3.18,0.48,9.5
2,white,6.0,0.26,0.29,3.1,0.041,37.0,144.0,0.98944,3.22,0.39,12.8
3,white,7.2,0.21,0.36,15.7,0.045,68.0,183.0,0.99922,3.25,0.76,9.4
4,white,6.5,0.18,0.33,1.4,0.029,35.0,138.0,0.99114,3.36,0.60,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...
5517,white,6.3,0.24,0.29,13.7,0.035,53.0,134.0,0.99567,3.17,0.38,10.6
5518,red,6.9,0.36,0.25,2.4,0.098,5.0,16.0,0.99640,3.41,0.60,10.1
5519,red,13.4,0.27,0.62,2.6,0.082,6.0,21.0,1.00020,3.16,0.67,9.7
5520,red,8.9,0.40,0.51,2.6,0.052,13.0,27.0,0.99500,3.32,0.90,13.4


In [6]:
x_test

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,white,7.0,0.250,0.45,2.3,0.045,40.0,118.0,0.99064,3.16,0.48,11.9
1,white,7.6,0.140,0.74,1.6,0.040,27.0,103.0,0.99160,3.07,0.40,10.8
2,white,6.2,0.150,0.27,11.0,0.035,46.0,116.0,0.99602,3.12,0.38,9.1
3,white,6.7,0.160,0.32,12.5,0.035,18.0,156.0,0.99666,2.88,0.36,9.0
4,white,6.8,0.270,0.22,17.8,0.034,16.0,116.0,0.99890,3.07,0.53,9.2
...,...,...,...,...,...,...,...,...,...,...,...,...
970,white,5.9,0.290,0.25,12.0,0.057,48.0,224.0,0.99810,3.23,0.41,9.0
971,white,7.4,0.260,0.30,7.9,0.049,38.0,157.0,0.99630,3.13,0.48,8.9
972,white,6.6,0.360,0.28,6.1,0.029,12.0,93.0,0.99054,3.19,0.27,12.8
973,red,7.7,0.715,0.01,2.1,0.064,31.0,43.0,0.99371,3.41,0.57,11.8


In [7]:
x_train=x_train.drop(['type'],axis=1)
x_test=x_test.drop(['type'],axis=1)

In [8]:
x_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,6.8,0.43,0.26,5.2,0.043,40.0,176.0,0.99116,3.17,0.41,12.3
1,6.4,0.27,0.45,8.3,0.050,52.0,196.0,0.99550,3.18,0.48,9.5
2,6.0,0.26,0.29,3.1,0.041,37.0,144.0,0.98944,3.22,0.39,12.8
3,7.2,0.21,0.36,15.7,0.045,68.0,183.0,0.99922,3.25,0.76,9.4
4,6.5,0.18,0.33,1.4,0.029,35.0,138.0,0.99114,3.36,0.60,11.5
...,...,...,...,...,...,...,...,...,...,...,...
5517,6.3,0.24,0.29,13.7,0.035,53.0,134.0,0.99567,3.17,0.38,10.6
5518,6.9,0.36,0.25,2.4,0.098,5.0,16.0,0.99640,3.41,0.60,10.1
5519,13.4,0.27,0.62,2.6,0.082,6.0,21.0,1.00020,3.16,0.67,9.7
5520,8.9,0.40,0.51,2.6,0.052,13.0,27.0,0.99500,3.32,0.90,13.4


# 3) 데이터 전처리
* 디펜스 코드와의 차이점: MinMaxScaler -> StandardScaler

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [10]:
set(train_y['quality'])

{3, 4, 5, 6, 7, 8, 9}

In [11]:
train_y = train_y['quality']-3

In [12]:
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(train_y.values)
x_test=torch.FloatTensor(x_test)
x_test

tensor([[-0.1694, -0.5446,  0.9076,  ..., -0.3676, -0.3523,  1.1883],
        [ 0.2917, -1.2080,  2.9041,  ..., -0.9282, -0.8811,  0.2637],
        [-0.7842, -1.1477, -0.3317,  ..., -0.6167, -1.0133, -1.1652],
        ...,
        [-0.4768,  0.1189, -0.2628,  ..., -0.1807, -1.7404,  1.9448],
        [ 0.3686,  2.2599, -2.1217,  ...,  1.1897,  0.2427,  1.1043],
        [ 1.5214,  3.4963, -0.5382,  ...,  1.6257,  0.0444, -0.5768]])

In [13]:
y_train = y_train.view([-1,1])
y_train

tensor([[3],
        [2],
        [4],
        ...,
        [3],
        [4],
        [5]])

In [14]:
y_train
y_one_hot = torch.zeros(5522, 7)
y_one_hot.scatter_(1, y_train, 1)
y_one_hot

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

# 4) 학습 모델 설정
* 디펜스 코드와의 차이점: layer 개수를 7개로 늘림, Dropout 함수 사용

In [15]:
lr = 0.001
training_epochs = 90
batch_size = 200
drop_prob = 0.01

In [16]:
from torch.utils.data import TensorDataset as TensorDataSet
dataset=TensorDataSet(x_train,y_one_hot)
data_loader=torch.utils.data.DataLoader(dataset=dataset,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       drop_last=True)

In [17]:
linear1 = torch.nn.Linear(11,256,bias=True)
linear2 = torch.nn.Linear(256,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,512,bias=True)
linear6 = torch.nn.Linear(512,512,bias=True)
linear7=torch.nn.Linear(512,7,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [18]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)
torch.nn.init.xavier_uniform_(linear7.weight)

Parameter containing:
tensor([[ 0.0789, -0.0885, -0.0239,  ...,  0.0524,  0.0567, -0.0500],
        [ 0.0720, -0.0752, -0.0753,  ..., -0.0045, -0.0189, -0.0079],
        [ 0.0329, -0.0647,  0.0425,  ..., -0.0261, -0.0711, -0.0439],
        ...,
        [-0.1059, -0.0250,  0.0657,  ..., -0.0383,  0.0840,  0.0515],
        [ 0.0557,  0.0117, -0.0397,  ...,  0.0372,  0.0890,  0.0083],
        [-0.0404, -0.0780, -0.0793,  ...,  0.0376, -0.0788, -0.0409]],
       requires_grad=True)

In [19]:
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                           linear3,relu,dropout,
                           linear4, relu, dropout,
                           linear5, relu, dropout,
                           linear6, relu, dropout,
                           linear7)

model=model.cuda()
optimizer= optim.Adam(model.parameters(),lr=lr)
loss= nn.CrossEntropyLoss().cuda()

In [20]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X=X.cuda()
        Y=Y.cuda()

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

       
    if epoch%10==0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(cost))
        

Epoch: 0001 cost = 1.116576314
Epoch: 0011 cost = 0.945833921
Epoch: 0021 cost = 0.591025531
Epoch: 0031 cost = 0.250790447
Epoch: 0041 cost = 0.193828195
Epoch: 0051 cost = 0.069052726
Epoch: 0061 cost = 0.103759728
Epoch: 0071 cost = 0.039914373
Epoch: 0081 cost = 0.048084144


# 5) Test data 예측

In [21]:
with torch.no_grad():
    x_test = x_test.cuda()
    prediction = model(x_test)
    
correct_prediction = torch.argmax(prediction, 1) 
correct_prediction

tensor([4, 3, 3, 3, 2, 5, 3, 2, 2, 3, 2, 3, 3, 2, 2, 4, 4, 3, 4, 2, 3, 2, 2, 3,
        4, 4, 3, 2, 1, 3, 4, 2, 2, 3, 3, 3, 3, 3, 5, 3, 2, 3, 3, 3, 2, 2, 3, 3,
        2, 2, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2, 3, 2, 4, 3, 3, 2,
        2, 2, 3, 2, 3, 2, 3, 4, 3, 2, 3, 3, 2, 2, 3, 2, 2, 3, 2, 3, 2, 3, 4, 2,
        5, 4, 3, 3, 3, 4, 2, 3, 2, 3, 3, 2, 2, 3, 2, 2, 4, 2, 3, 2, 3, 2, 3, 2,
        2, 3, 2, 1, 3, 3, 2, 2, 2, 2, 5, 4, 3, 2, 4, 3, 2, 4, 3, 2, 3, 3, 2, 2,
        2, 2, 3, 3, 2, 2, 2, 3, 4, 4, 3, 3, 3, 2, 2, 4, 2, 3, 3, 2, 4, 4, 2, 3,
        3, 3, 4, 5, 4, 3, 3, 4, 3, 3, 3, 2, 3, 4, 2, 2, 3, 2, 3, 2, 3, 3, 2, 5,
        3, 3, 3, 3, 4, 2, 2, 4, 4, 2, 2, 5, 4, 1, 2, 4, 2, 3, 3, 3, 3, 4, 3, 3,
        2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2, 5, 2, 3, 5, 2, 2, 2, 3, 4, 3,
        3, 4, 4, 3, 2, 3, 3, 2, 4, 4, 2, 2, 4, 3, 2, 3, 2, 4, 2, 4, 4, 2, 3, 1,
        2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 4, 3, 3, 1, 2, 1, 4, 3, 2, 3, 3, 3, 3,
        3, 3, 4, 2, 3, 3, 3, 2, 4, 2, 3,

In [22]:
submit['quality'] = correct_prediction.cpu().detach()
submit['quality'] = submit['quality']+3
submit

,index,quality
0,0,7
1,1,6
2,2,6
3,3,6
4,4,5
...,...,...
970,970,6
971,971,6
972,972,7
973,973,6


In [23]:
submit.to_csv('submit.csv',index = None)